In [1]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
import string

In [2]:
from neo4j import GraphDatabase

uri = "bolt://192.168.1.203:7687"

driver = GraphDatabase.driver(uri, auth=('neo4j', 'oscarfelix'), encrypted=False)


In [3]:
with driver.session() as session:
    result = session.run("MATCH (a) RETURN a limit 5")

In [4]:
result.data()

[{'a': <Node id=0 labels={'Movie'} properties={'IMDb_Url': 'https://www.imdb.com/title/tt0086543/?ref_=ref_ext_clz', 'IMDb_Rating': 7.2, 'Color': 'Color', 'Plot': 'An alcoholic writer begins having visions apparently warning him of impending danger. After he delivers a lecture, he begins an affair with a woman from the audience. The visions intensify, however, and begin to threaten his relationship with the woman, since it may be her the visions are warning against.', 'Director': 'Paul Verhoeven', 'Title': 'The 4th Man', 'Release_Year': 1983, 'Studio': 'Rob Houwer Productions', 'Name': 'The 4th Man', 'Photography': 'Jan de Bont', 'Runtime': 102, 'Producer': 'Rob Houwer', 'Language': 'Dutch', 'Musician': 'Loek Dikker', 'Release_Date': 1983, 'Actor': 'Jeroen Krabbé, Renée Soutendijk, Thom Hoffman', 'Country': 'The Netherlands', 'Genre': 'Drama, Mystery, Thriller', 'Writer': 'Gerard Reve, Gerard Soeteman'}>},
 {'a': <Node id=1 labels={'Movie'} properties={'IMDb_Url': 'https://www.imdb.com

In [5]:
df = pd.read_csv("export_movies.csv")

In [6]:
df.head()

,Title,Genre,Actor,Director,Musician,Photography,Producer,Writer,Release Year,Studio,Release Date,Language,Country,Color,Runtime,Region,IMDb Url,Plot,IMDb Rating,Audience Rating
0,The 4th Man,"Drama, Mystery, Thriller","Jeroen Krabbé, Renée Soutendijk, Thom Hoffman",Paul Verhoeven,Loek Dikker,Jan de Bont,Rob Houwer,"Gerard Reve, Gerard Soeteman",1983,Rob Houwer Productions,1983,Dutch,The Netherlands,Color,102,NaN,https://www.imdb.com/title/tt0086543/?ref_=ref...,An alcoholic writer begins having visions appa...,7.2,NaN
1,The Adventure of Denchu-Kozo,"Comedy, Horror, Science Fiction","Kei Fujiwara, Nobu Kanaoka, N. Senba",Shin'ya Tsukamoto,NaN,NaN,NaN,Shin'ya Tsukamoto,1987,NaN,1987,Japanese,Japan,Color,47,NaN,https://www.imdb.com/title/tt0166069/?ref_=ref...,Hikari is a boy who is bullied and teased by t...,6.6,NaN
2,Alien from L.A.,"Adventure, Comedy, Science Fiction","Kathy Ireland, William R. Moses, Richard Haines",Albert Pyun,"Jim Andron, Steve LeGassick, Anthony Riparetti",Tom Fraser,"Yoram Globus, Menahem Golan, Tom Karnowski","Debra Ricci, Sandra Berg, Albert Pyun",1988,Golan-Globus Productions,1988,English,USA,Color,87,NaN,https://www.imdb.com/title/tt0092532/?ref_=ref...,When her archaeologist father disappears on an...,2.9,US - PG
3,Alley Cat,Action,"Karin Mani, Robert Torti, Britt Helfer","Victor M. Ordonez, Ed Palmos, Al Valletta",Quito Colayco,Howard A. Anderson III,"Claudia Decea, David A. Gronsky, Karin Mani",Robert E. Waters,1984,Multicom Entertainment Group,1984,English,USA,Color,68,Region A,https://www.imdb.com/title/tt0083541/?ref_=ref...,A beautiful young martial arts expert stops a ...,5.5,US - R
4,Are You in the House Alone?,"Drama, Horror, Thriller","Kathleen Beller, Blythe Danner, Tony Bill",Walter Grauman,Charles Bernstein,Jack Swain,"Jay Benson, Richard Berg, Charles W. Fries","Judith Parker, Richard Peck",1978,CBS,1978,English,USA,Color,96,NaN,https://www.imdb.com/title/tt0077174/?ref_=ref...,Teenage girl is plagued by harassing phone cal...,5.6,NaN


In [7]:
df = df.fillna('null')

In [8]:
with driver.session() as session:
    result = session.run("Match ()-[e]-() delete e")
    result = session.run("Match (n)delete n")
    result = session.run("MATCH (a) RETURN a limit 5")
result.data()

[]

In [9]:
row_dicts = df.to_dict(orient='records')

In [10]:
# row_dicts

In [11]:
# CREATE (TheMatrix:Movie {title:'The Matrix', released:1999, tagline:'Welcome to the Real World'})
# CREATE (Keanu:Person {name:'Keanu Reeves', born:1964})
# CREATE (Carrie:Person {name:'Carrie-Anne Moss', born:1967})
# CREATE (Laurence:Person {name:'Laurence Fishburne', born:1961})
# CREATE (Hugo:Person {name:'Hugo Weaving', born:1960})
# CREATE (LillyW:Person {name:'Lilly Wachowski', born:1967})
# CREATE (LanaW:Person {name:'Lana Wachowski', born:1965})
# CREATE (JoelS:Person {name:'Joel Silver', born:1952})

In [12]:
def node_feat(row):
    pairstrings = []
    for x in row.items():
        feat = x[0].replace(" ","_")
        val = x[1]
        if x[1] == 'null':
            strin = f"{feat}:{val}"
        elif isinstance(x[1], str):
            val = val.replace('"',"")
            strin = f'{feat}:"{val}"'
        else:
            strin = f"{feat}:{val}"

        pairstrings.append(strin)

    dictstring = ", ".join(pairstrings)
    return dictstring
    

In [13]:
def create_movie( movie):
    return "CREATE (%s:Movie {%s, %s})" % (
        movie['Title'].lower().translate(str.maketrans('', '', string.punctuation)).replace(" ","_"),
        f"""name:"{movie['Title']}" """, 
        node_feat(movie))

In [14]:
for row in row_dicts:
    with driver.session() as sess:
        sess.run(create_movie(row))

In [15]:
create_movie(row_dicts[0])

'CREATE (the_4th_man:Movie {name:"The 4th Man" , Title:"The 4th Man", Genre:"Drama, Mystery, Thriller", Actor:"Jeroen Krabbé, Renée Soutendijk, Thom Hoffman", Director:"Paul Verhoeven", Musician:"Loek Dikker", Photography:"Jan de Bont", Producer:"Rob Houwer", Writer:"Gerard Reve, Gerard Soeteman", Release_Year:1983, Studio:"Rob Houwer Productions", Release_Date:1983, Language:"Dutch", Country:"The Netherlands", Color:"Color", Runtime:102, Region:null, IMDb_Url:"https://www.imdb.com/title/tt0086543/?ref_=ref_ext_clz", Plot:"An alcoholic writer begins having visions apparently warning him of impending danger. After he delivers a lecture, he begins an affair with a woman from the audience. The visions intensify, however, and begin to threaten his relationship with the woman, since it may be her the visions are warning against.", IMDb_Rating:7.2, Audience_Rating:null})'

In [16]:
with driver.session() as session:
    result = session.run("MATCH (a) RETURN count(a)")

In [17]:
result.data()

[{'count(a)': 98}]

In [18]:
def people_list(row, col):
    plist = row[col].split(", ")
    return plist


In [19]:
def merge_people_query(col, edge, num_peeps):  
    
    mmatch = "MATCH (m:Movie {Title:$movie})"
    pmatch = [mmatch]
    attach = []
    peeps = [f"person_{x}" for x in range(num_peeps)]
    
    for peep in peeps:
        pmatch.append("MERGE (%s:Person {name:%s})" % (peep, "$"+peep))
        attach.append("MERGE (m)<-[:%s]-(%s)" % (edge, peep))
        attach.append("SET %s : %s" % (peep, col))
    
    merges = " \n".join(pmatch + attach)
    return merges


In [20]:
print(merge_people_query('Actor', 'ACTED_IN', 4))

MATCH (m:Movie {Title:$movie}) 
MERGE (person_0:Person {name:$person_0}) 
MERGE (person_1:Person {name:$person_1}) 
MERGE (person_2:Person {name:$person_2}) 
MERGE (person_3:Person {name:$person_3}) 
MERGE (m)<-[:ACTED_IN]-(person_0) 
SET person_0 : Actor 
MERGE (m)<-[:ACTED_IN]-(person_1) 
SET person_1 : Actor 
MERGE (m)<-[:ACTED_IN]-(person_2) 
SET person_2 : Actor 
MERGE (m)<-[:ACTED_IN]-(person_3) 
SET person_3 : Actor


In [21]:
def people_args(row, col):
    pl = people_list(row, col)
    peeps = [f"person_{x}" for x in range(len(pl))]
    peep_dict = dict(zip(peeps, pl))
    return peep_dict

In [22]:
def merge_people(driver, row, col, edge):
    
    num_people = len(people_list(row,col))
    query_template = merge_people_query(col, edge, num_people)
    peep_dict = people_args(row, col)
    
    with driver.session() as session:
        result = session.run(query_template,
                             movie=row['Title'],
                             **peep_dict
                            )

In [23]:
people_cols = ['Actor','Director', 'Musician', 'Photography', 'Producer', 'Writer']
edges = ["ACTED_IN", "DIRECTED", "MUSCIAN_IN", "PHOTOGRAPHY_FOR", "PRODUCED", "WROTE"]
role_zip = list(zip(people_cols, edges))
for row in row_dicts:
    for role in role_zip:
        merge_people(driver, row, role[0], role[1])


In [24]:
# think about paralells edges

In [ ]:
# Genre',"IN_GENRE",

In [28]:
# with driver.session() as session:
#     result = session.run("MATCH ()-[e]-() RETURN e")

In [30]:
# result.data()